In [132]:
#########################
## Mark Bjerregaard     #
## MARKBJ@UMICH.EDU     #
#########################

import time
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
from bs4 import BeautifulSoup
import requests
import time
import json
import re
from datetime import date


'''
Linkedin job scrape script
Scrapes jobs using selenium and chromedriver available here: https://chromedriver.chromium.org/

Steps:
1. Enter Linkedin log-in page
2. Spot the cookies pop-up and accept cookies
3. Fill E-Mail Adress and Password areas and click login
4. Click on the jobs from the section above
5. Search for job positions Data Analyst 
6. Scroll till end of page, collecting links on the way
7. Go to the next page when it is the end of the page while keep collecting links
8. After all links are collected, go to each link
9. Click the see more button to expand the job description text
10. Scrape the desired data


Items to be scraped 
1. Job title 
2. Company name
3. Company location
4. job description
5. work method (hybrid, remote, on-site)
6. Post date
'''
''

def open_cache(cache_name):
    '''
    opens the cache file if it exists and loads the JSON into a dictionary, which it then returns
    If the cahce file doesn't exist, creates a new chace dictionary
    Parameters
    ---
    None 
    Returns
    ---
    The opened cache
    '''
    try:
        cache_file = open(cache_name, 'r')
        cache_contents = cache_file.read()
        cache_dict = json.loads(cache_contents)
        cache_file.close()
    except:
        cache_dict = {}
    return cache_dict

def save_cache(cache_dict, cache_name):
    '''
    saves the current state of the cache to disk
    Parameters
    ---
    cache_dict: dict
        the dictionary to save
    Returns
    ---
    None
    '''
    dumped_json_cache = json.dumps(cache_dict, indent=4)
    fw = open(cache_name,'w')
    fw.write(dumped_json_cache)
    fw.close()


In [ ]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json


page_iterations = 100
search_area = 'Copenhagen Metropolitan Area'
search_position = 'Data Analyst'

def open_cache(cache_name):
    '''
    opens the cache file if it exists and loads the JSON into a dictionary, which it then returns
    If the cahce file doesn't exist, creates a new chace dictionary
    Parameters
    ---
    None 
    Returns
    ---
    The opened cache
    '''
    try:
        cache_file = open(cache_name, 'r')
        cache_contents = cache_file.read()
        cache_dict = json.loads(cache_contents)
        cache_file.close()
    except:
        cache_dict = {}
    return cache_dict

def save_cache(cache_dict, cache_name):
    '''
    saves the current state of the cache to disk
    Parameters
    ---
    cache_dict: dict
        the dictionary to save
    Returns
    ---
    None
    '''
    dumped_json_cache = json.dumps(cache_dict, indent=4)
    fw = open(cache_name,'w')
    fw.write(dumped_json_cache)
    fw.close()

def job_scrape(driver):
    '''
    Function for scraping page of job links
    Input: chromedriver (current webpage)
    output: list of job site links
    ''' 
    links_cache = 'link_cache.json'
    link_dict = open_cache(links_cache)

    link_count = max(list(link_dict.values()))
    jobs_list = driver.find_elements(By.CLASS_NAME, 'job-box')
    for job in jobs_list:
        link = job.get_attribute('href')
        if not link_dict.get(link):
            link_dict[job.get_attribute('href')] = link_count + 1
            link_count += 1 

    save_cache(link_dict, links_cache)

    return link_dict

# Chrome driver setup & browser open
options = Options()
options.add_argument('start-maximized')
driver = webdriver.Chrome(service =Service('chromedriver.exe'), options=options)
driver.implicitly_wait(10)

# Open page
driver.get('https://graduateland.com/jobs')
time.sleep(2)

# Filtering on area
driver.find_element(By.XPATH, '//*[@id="search-filters"]/div[46]/div/div[1]/input').send_keys(search_area)
driver.find_element(By.XPATH, '//*[@id="search-filters"]/div[46]/div/div[1]/input').click()
driver.find_element(By.XPATH, '//*[@id="search-filters"]/div[46]/div/div[2]/label[2]').click()

# Entering position filter keywords
driver.find_element(By.XPATH, '//*[@id="job-search-form"]/div[1]/div[1]/div[1]/input').send_keys(search_position, Keys.ENTER)
time.sleep(3)

#initializing scrape loop at page 1
page = 1 
print('scraping job links, page:', page)
links = job_scrape(driver)

# Page iterator
for i in range(1, page_iterations):
    # handling XPATH variations based on page numbers
    if i < 3:
        index = i * 2 
    elif i < 6:
        index = i + 2
    else:
        i = 7
    # Clicking next page button
    driver.find_element(By.XPATH, '//*[@id="timeline"]/div[3]/div/a[{}]'.format(index)).click()
    page += 1
    print('scraping job links, page:', page)
    time.sleep(3)
    # Scraping job post links from page
    links = job_scrape(driver)
    if i % 20 == 0:
        time.sleep(60)



In [147]:
from bs4 import BeautifulSoup
import requests
import time
import json
import re
from datetime import date




def Offline_test(soup, source):
    #returns True of job is offline
    offline = soup.find('section', class_='job-offline')
    if offline == None:
        return False
    else:
        links_cache = 'link_cache.json'
        link_dict = open_cache(links_cache)
        del link_dict[source]
        save_cache(link_dict, links_cache)
        return True

def Scraper(url):
    
    def attribute_extract(soup):
        attributes = soup.find('div', class_='content-description')
        attributes = attributes.find_all('p')
        last = ''
        attributes_list = [i.text.strip() for i in attributes if i.text != None]
        idx = 0
        for i in attributes_list:
            new_elem = re.sub('  ', '', i)
            new_elem = re.sub('\\n\\n\\n','___',new_elem)
            new_elem = new_elem.split('___')
            if len(new_elem) != 1:
                new_elem = [re.sub('\\n', '', i) for i in new_elem]
                attributes_list[idx] = new_elem
            else:
                new_elem = re.sub('\\n',' ',new_elem[0])
                attributes_list[idx] = new_elem
            idx += 1 
        return attributes_list


    vertex_cache = 'vertex_cache.json'
    vert_dict = open_cache(vertex_cache)
    if url in vert_dict:
        attr_dict = vert_dict[url]
        return attr_dict
    else:
        attr_dict = {}



    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    

    # Title 
    title_ = soup.find('div',class_='job-title').find('h1').text
    attr_dict['title'] = title_

    # Job description
    job_desc_ = soup.find('article', class_='box-item job-content').text
    attr_dict['job_desc'] = job_desc_


    headline = soup.find('div', class_='headline')
    # Company
        
    company_ = headline.find('h2').text.strip()
    attr_dict['company'] = company_

    # Industry
    industry_ = headline.find_all('p')[0].text.strip()
    attr_dict['industry'] = industry_
    
    # Followers
    followers_ = headline.find_all('p')[1].text.strip()
    followers_ = re.sub('[^0-9]', '', followers_)
    attr_dict['followers'] = followers_

    #logo
    try:
        logo_soup = soup.find('div',class_='company-item')
        logo_ = logo_soup.find('img')['src']
        attr_dict['logo'] = logo_ 
    except:
        attr_dict['logo'] = None

    # Attributes (location, category, job_type, skills, language)
    attributes = attribute_extract(soup)
    location_ = attributes[0]
    category_ = attributes[1]
    job_type_ = attributes[2]
    skills_ = attributes[3]
    language_ = attributes[4]
    attr_dict['location'] = location_
    attr_dict['category'] = category_
    attr_dict['job_type'] = job_type_
    attr_dict['skills'] = skills_
    attr_dict['language'] = language_

    vert_dict[url] = attr_dict
    save_cache(vert_dict, vertex_cache)
    return attr_dict


class Vertex:
    def __init__(self, url):

        attr_dict = Scraper(url)
        self.attr_dict = attr_dict
        self.title = attr_dict.get('title')
        self.blob = attr_dict.get('job_desc')
        self.company = attr_dict.get('company')
        self.industry = attr_dict.get('industry')
        self.followers = attr_dict.get('followers')
        self.logo = attr_dict.get('logo')
        self.location = attr_dict.get('location')
        self.category = attr_dict.get('category')
        self.job_type = attr_dict.get('job_type')
        self.skills = attr_dict.get('skills')
        self.language_req = attr_dict.get('language')
        self.day_extracted = date.today()

        


vert_test = Vertex('https://graduateland.com/job/51447115/13')


In [149]:
dict = vert_test.attr_dict
pretty = json.dumps(dict, indent=4)
print(pretty)

{
    "title": "Student assistant for supporting leaders in their daily tasks at \u00d8rsted",
    "job_desc": "\nImagine a future where you use your ability to keep an overview and your coordination skills to support leaders in their daily responsibilities.Join us in this role where you\u2019ll be responsible for administrative tasks related to our Danish leaders\u2019 work. You\u2019ll have onboarding tasks, such as ordering equipment, ID cards, and gifts and you\u2019ll ensure that all pre-onboarding related tasks are done before hiring date. You\u2019ll also be responsible for having an overview of all external consultants and, e.g., their extensions and termination. You\u2019ll furthermore be supporting leaders in conducting yearly recurring people leadership activities.Welcome to ITYou\u2019ll be part of People Leadership Centre of Excellence (CoE) within IT where you, together with your colleagues, will build and maintain a strong people leadership capability for the IT organisa

In [143]:

url = 'https://graduateland.com/job/51447115/13'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')
test = soup.find('div',class_='job-title').find('h1')

print(test.text)

Student assistant for supporting leaders in their daily tasks at Ørsted


In [127]:
from bs4 import BeautifulSoup
import requests
import time
import json
import re

#BASE_URL = 'https://graduateland.com/job/51447115/13'
BASE_URL = 'https://graduateland.com/job/51447115/13'

response = requests.get(BASE_URL)
soup = BeautifulSoup(response.text,'html.parser')

##################################################################
#job offline test
def Offline_test(soup, source):
    #returns True of job is offline
    offline = soup.find('section', class_='job-offline')
    if offline == None:
        return False
    else:
        links_cache = 'link_cache.json'
        link_dict = open_cache(links_cache)
        del link_dict[source]
        save_cache(link_dict, links_cache)
        return True


# Job description
job_desc = soup.find('article', class_='box-item job-content').text
# Company
headline = soup.find('div', class_='headline') 
company = headline.find('h2').text.strip()
# Industry
industry = headline.find_all('p')[0].text.strip()
# Followers
followers = headline.find_all('p')[1].text.strip()
followers = re.sub('[^0-9]', '', followers) 
#Expiration
try:
    expiration = soup.find('span', class_= 'text-warning').text.strip()
except: 
    expiration = None

#logo
logo_soup = soup.find('div',class_='company-item')
logo = logo_soup.find('img')['src']

# Attributes
attributes = soup.find('div', class_='content-description')
attributes = attributes.find_all('p')

def attribute_extract(soup):
    attributes = soup.find('div', class_='content-description')
    attributes = attributes.find_all('p')
    last = ''
    attributes_list = [i.text.strip() for i in attributes if i.text != None]
    idx = 0
    for i in attributes_list:
        new_elem = re.sub('  ', '', i)
        new_elem = re.sub('\\n\\n\\n','___',new_elem)
        new_elem = new_elem.split('___')
        if len(new_elem) != 1:
            new_elem = [re.sub('\\n', '', i) for i in new_elem]
            attributes_list[idx] = new_elem
        else:
            new_elem = re.sub('\\n',' ',new_elem[0])
            attributes_list[idx] = new_elem
        idx += 1 
    return attributes_list

attributes = attribute_extract(soup)
location = attributes[0]
category = attributes[1]
job_type = attributes[2]
skills = attributes[3]
language = attributes[4]




print(company)
print(industry)
print(followers)
print(expiration)
print(logo)
print(location)
print(category)
print(skills)
print(language)


Ørsted
Multiple industries
4062
Expires in 5 days
https://d1k976m6pd0u9m.cloudfront.net/public/employer/logo/g43ql0iq55__rsted.png.webp
Gentofte, Denmark
Administration
Part-time
['Team Player', ' Structured', ' Coordination']


In [130]:

BASE_URL = 'https://graduateland.com/project/45825207/13'
BASE_URL = 'https://graduateland.com/job/51447115/13'
response = requests.get(BASE_URL)
soup = BeautifulSoup(response.text,'html.parser')

attributes = soup.find('div', class_='content-description')
attributes = attributes.find_all('p')

last = ''
attributes_list = [i.text.strip() for i in attributes if i.text != None]
idx = 0
for i in attributes_list:
    new_elem = re.sub('  ', '', i)
    new_elem = re.sub('\\n\\n\\n','___',new_elem)
    new_elem = new_elem.split('___')
    if len(new_elem) != 1:
        new_elem = [re.sub('\\n', '', i) for i in new_elem]
        attributes_list[idx] = new_elem
    else:
        new_elem = re.sub('\\n',' ',new_elem[0])
        attributes_list[idx] = new_elem
    idx += 1 
#print(attributes_list)
location = attributes_list[0]
category = attributes_list[1]
job_type = attributes_list[2]
skills = attributes_list[3]

language = attributes_list[4]

print(location)
print(category)
print(job_type)
print(skills)
print(language)

#print(list1)    
    
#for i in features2:
#    print(i.text.strip())


Gentofte, Denmark
Administration
Part-time
['Team Player', ' Structured', ' Coordination']
English, Professional


In [21]:
i = i.strip()
    for j in i:
        print(j)
        if (j == ' ' and last == ' ') or (j == 'n' and last == '\\'):
            continue
        else: 
            new_elem.append(j)
        last = j
    list1[idx] = ''.join(new_elem)
    idx += 1 
print(list1)         


1


In [63]:
from bs4 import BeautifulSoup
import requests
import time
import json


BASE_URL = 'https://graduateland.com/project/51506904/13'
CACHE_DICT = {}

headers = {'User-Agent': 'UMSI 507 Course Project - Python Web Scraping','From': 'youremail@domain.com','Course-Info': 'https://www.si.umich.edu/programs/courses/507'}

##### Make Soup for courses page



def load_cache():
    try:
        cache_file = open(CACHE_FILE_NAME, 'r')
        cache_file_contents = cache_file.read()
        cache = json.loads(cache_file_contents)
        cache_file.close()
    except:
        cache = {}
    return cache


def save_cache(cache):
    cache_file = open(CACHE_FILE_NAME, 'w')
    contents_to_write = json.dumps(cache)
    cache_file.write(contents_to_write)
    cache_file.close()


def make_url_request_using_cache(url, cache):
    if (url in cache.keys()): # the url is our unique key
        print("Using cache")
        return cache[url]
    else:
        print("Fetching")
        time.sleep(1)
        response = requests.get(url, headers=headers)
        cache[url] = response.text
        save_cache(cache)
        return cache[url]


# Load the cache, save in global variable
CACHE_DICT = load_cache()

courses_page_url = BASE_URL + COURSES_PATH
url_text = make_url_request_using_cache(courses_page_url, CACHE_DICT)
#response = requests.get(courses_page_url)

soup = BeautifulSoup(response.text, 'html.parser')

Sparekassen Sjælland-Fyn
Banking & Finance
2


In [65]:
company = soup.find('div', class_='headline').find_all('p')
print(list(str(company[1].text.strip())))


['2', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'f', 'o', 'l', 'l', 'o', 'w', 'e', 'r', 's']


In [27]:
course_desc = soup.find('article', class_='box-item job-content')
print(len(course_desc))
print(course_desc.text)

46

Passioneret servicehaj som 1. assistentHvad slår en passioneret servicehaj, der lige hurtigt viser dig vej til ugens spottilbud? Ikke ret meget, hvis du spørger os. Og hvis du er enig, så er denne stilling måske noget for dig. Som 1. assistent finder man dig nemlig overalt i butikken. Her er det dit ansvar, at varehylderne spiller, og at butikken ser præsentabel ud. Der er aldrig to dage, der er ens, og vi regner med dig – ligesom at du altid kan regne med os. Helt kort så skal dualtid levere den bedste kundeserviceekspedere kunder ved kassentrimme hylder og fylde varer oppåtage dig lederansvar, når det kræves og fx lukke butikken de dage, hvor butikschefen ikke er derAlle de gode grundeSom 1. assistent i Netto bliver du en del af en sjov arbejdsplads med fantastiske kollegaer, godt fællesskab og mange forskellige slags opgaver. Og hvis vi lige skal nævne et par andre ting, så får du også frisk frugt og grønt på arbejdspladsen – hver dagfoden ind hos Salling Group og et kæmpe karri

In [13]:
course_save = course_desc 

In [20]:
BASE_URL = 'https://graduateland.com/job/51498185/23/1-assistent-gentofte'

response = requests.get(BASE_URL)
soup = BeautifulSoup(response.text,'html.parser')
course_desc = soup.find_all('article', class_='box-item job-content')
print(len(course_desc))
print(course_desc[0].text.strip())


1
Passioneret servicehaj som 1. assistentHvad slår en passioneret servicehaj, der lige hurtigt viser dig vej til ugens spottilbud? Ikke ret meget, hvis du spørger os. Og hvis du er enig, så er denne stilling måske noget for dig. Som 1. assistent finder man dig nemlig overalt i butikken. Her er det dit ansvar, at varehylderne spiller, og at butikken ser præsentabel ud. Der er aldrig to dage, der er ens, og vi regner med dig – ligesom at du altid kan regne med os. Helt kort så skal dualtid levere den bedste kundeserviceekspedere kunder ved kassentrimme hylder og fylde varer oppåtage dig lederansvar, når det kræves og fx lukke butikken de dage, hvor butikschefen ikke er derAlle de gode grundeSom 1. assistent i Netto bliver du en del af en sjov arbejdsplads med fantastiske kollegaer, godt fællesskab og mange forskellige slags opgaver. Og hvis vi lige skal nævne et par andre ting, så får du også frisk frugt og grønt på arbejdspladsen – hver dagfoden ind hos Salling Group og et kæmpe karrier